🇺🇸Using the dataset RVF_Project.xlsx, generate the code with explanatory hashtags in both English and Portuguese for calculating RFM (Recency, Frequency, Monetary Value). Create plots with a dark background and turquoise color scheme.
Next, create scoring bands from 1 to 5 for recency, frequency, and monetary value. I provided pivot tables as reference, but you can perform all calculations in Python if preferred.
The final deliverable, as illustrated in the 'Resposta' sheet, should include each customer's corresponding recency, frequency, and monetary value bands. Also, provide a table showing the cut-off points used for each band.



🇧🇷 Usando o dataset RVF_Project.xlsx, gere o código com hashtags explicativas em inglês e português para calcular o RFM (Recência, Frequência, Valor Monetário). Crie gráficos com fundo escuro e esquema de cores turquesa.
Em seguida, crie faixas de pontuação de 1 a 5 para recência, frequência e valor monetário. Eu forneci tabelas dinâmicas como referência, mas você pode realizar todos os cálculos em Python, se preferir.

O produto final, conforme ilustrado na aba 'Resposta', deve incluir para cada cliente as respectivas faixas de recência, frequência e valor monetário. Além disso, forneça uma tabela com os pontos de corte utilizados para cada faixa.


In [ ]:
from google.colab import userdata
userdata.get('GOOGLE_API_KEY_1')

# Task
Analyze the customer data in "RVF_Project.xlsx" to calculate RFM (Recency, Frequency, Monetary Value), create 5-band scores for each RFM component, and present a table showing each customer's RFM bands along with the cut-off points used for the bands. Include explanatory code comments in both English and Portuguese and generate plots with a dark background and turquoise color scheme.

## Load the data

### Subtask:
Load the data from the "RVF_Project.xlsx" file into a pandas DataFrame.


**Reasoning**:
Import the pandas library and load the data from the Excel file into a DataFrame. Display the first few rows to confirm the data is loaded correctly.



In [ ]:
# Import pandas library / Importa a biblioteca pandas
import pandas as pd

# Load data from the Excel file into a DataFrame / Carrega os dados do arquivo Excel em um DataFrame
file_path = '/content/RVF_Project.xlsx'
df = pd.read_excel(file_path)

# Display the first few rows of the DataFrame / Exibe as primeiras linhas do DataFrame
display(df.head())

**Reasoning**:
The data was loaded, but it includes extra rows and columns that are not part of the actual data. Based on the column headers in row 1 and the data starting from row 2, I need to load the data again, skipping the first two rows and specifying the correct column names for 'Valor', 'Frequência', and 'Recência'.



In [ ]:
# Load data again, skipping the first two rows and specifying column names / Carrega os dados novamente, pulando as duas primeiras linhas e especificando os nomes das colunas
df = pd.read_excel(file_path, skiprows=2, names=['Valor_Cliente', 'Monetary', 'Unnamed_2', 'Unnamed_3', 'Frequencia_Cliente', 'Frequency', 'Unnamed_5', 'Unnamed_6', 'Recencia_Cliente', 'Recency'])

# Drop the rows that contain 'Total Geral' / Remove as linhas que contêm 'Total Geral'
df = df[df['Valor_Cliente'] != 'Total Geral'].copy()

# Select only the relevant columns for RFM / Seleciona apenas as colunas relevantes para RFM
df = df[['Valor_Cliente', 'Monetary', 'Frequency', 'Recency']]

# Display the first few rows of the cleaned DataFrame / Exibe as primeiras linhas do DataFrame limpo
display(df.head())

## Data preparation

### Subtask:
Clean and prepare the data for RFM analysis. This may involve handling missing values, converting data types, and selecting relevant columns.


**Reasoning**:
Inspect the data types and convert columns to the appropriate types for RFM analysis.



In [ ]:
# Inspect data types of the DataFrame columns / Inspeciona os tipos de dados das colunas do DataFrame
print("Data types before conversion:") # Imprime os tipos de dados antes da conversão
display(df.dtypes)

# Convert 'Recency' column to datetime / Converte a coluna 'Recency' para o tipo datetime
df['Recency'] = pd.to_datetime(df['Recency'], errors='coerce')

# Convert 'Monetary' and 'Frequency' columns to numeric, handling potential errors / Converte as colunas 'Monetary' e 'Frequency' para numérico, lidando com erros potenciais
df['Monetary'] = pd.to_numeric(df['Monetary'], errors='coerce')
df['Frequency'] = pd.to_numeric(df['Frequency'], errors='coerce')

# Check for and handle missing values that resulted from conversion / Verifica e lida com valores ausentes resultantes da conversão
print("\nMissing values after conversion:") # Imprime os valores ausentes após a conversão
display(df.isnull().sum())

# Drop rows with missing values if any resulted from coercion / Remove linhas com valores ausentes, se houver, resultantes da coerção
df.dropna(inplace=True)

# Inspect data types after conversion and dropping missing values / Inspeciona os tipos de dados após a conversão e remoção de valores ausentes
print("\nData types after conversion and dropping missing values:") # Imprime os tipos de dados após a conversão e remoção de valores ausentes
display(df.dtypes)

# Display the first few rows to confirm changes / Exibe as primeiras linhas para confirmar as mudanças
display(df.head())

## Calculate rfm

### Subtask:
Calculate Recency, Frequency, and Monetary Value for each customer.


**Reasoning**:
Determine the analysis date and calculate the Recency for each customer.



In [ ]:
import datetime as dt

# Determine the analysis date (a date slightly after the latest transaction)
# Determina a data de análise (uma data ligeiramente após a data da última transação)
analysis_date = df['Recency'].max() + dt.timedelta(days=1)
print(f"Analysis Date: {analysis_date}") # Imprime a data de análise

# Calculate Recency in days for each customer
# Calcula a Recência em dias para cada cliente
df['Recency_Days'] = (analysis_date - df['Recency']).dt.days

# Display the first few rows with the new 'Recency_Days' column
# Exibe as primeiras linhas com a nova coluna 'Recency_Days'
display(df[['Valor_Cliente', 'Recency_Days', 'Frequency', 'Monetary']].head())

## Create rfm bands

### Subtask:
Create scoring bands (1 to 5) for Recency, Frequency, and Monetary Value based on the calculated values.


**Reasoning**:
Define the number of bands and then use quantiles to create scoring bands for Recency, Frequency, and Monetary Value, assigning scores based on the instructions and adding them as new columns to the DataFrame.



In [ ]:
# Define the number of bands for scoring / Define o número de faixas para pontuação
num_bands = 5

# Create scoring bands for Recency / Cria faixas de pontuação para Recência
# Lower Recency_Days means higher recency, so score 5 is for the lowest Recency_Days (most recent)
# Menor Recency_Days significa maior recência, então a pontuação 5 é para o menor Recency_Days (mais recente)
df['R_Score'] = pd.qcut(df['Recency_Days'], num_bands, labels=[5, 4, 3, 2, 1])

# Create scoring bands for Frequency / Cria faixas de pontuação para Frequência
# Higher Frequency means higher score
# Maior Frequência significa maior pontuação
df['F_Score'] = pd.qcut(df['Frequency'], num_bands, labels=[1, 2, 3, 4, 5])

# Create scoring bands for Monetary Value / Cria faixas de pontuação para Valor Monetário
# Higher Monetary means higher score
# Maior Valor Monetário significa maior pontuação
df['M_Score'] = pd.qcut(df['Monetary'], num_bands, labels=[1, 2, 3, 4, 5])

# Display the first few rows with the new score columns / Exibe as primeiras linhas com as novas colunas de pontuação
display(df[['Valor_Cliente', 'Recency_Days', 'R_Score', 'Frequency', 'F_Score', 'Monetary', 'M_Score']].head())

## Combine rfm bands

### Subtask:
Combine the RFM bands to create a single RFM score or segment for each customer.


**Reasoning**:
Concatenate the RFM scores into a single string and create a composite RFM score.



In [ ]:
# Concatenate the R, F, and M scores into a single string to create the RFM segment
# Concatena as pontuações R, F e M em uma única string para criar o segmento RFM
df['RFM_Segment'] = df['R_Score'].astype(str) + df['F_Score'].astype(str) + df['M_Score'].astype(str)

# Create a composite RFM score by summing the individual scores
# Cria uma pontuação RFM composta somando as pontuações individuais
df['RFM_Score'] = df['R_Score'].astype(int) + df['F_Score'].astype(int) + df['M_Score'].astype(int)

# Display the first few rows of the DataFrame including the new RFM columns
# Exibe as primeiras linhas do DataFrame incluindo as novas colunas RFM
display(df[['Valor_Cliente', 'R_Score', 'F_Score', 'M_Score', 'RFM_Segment', 'RFM_Score']].head())

## Visualize rfm

### Subtask:
Create visualizations (with dark background and turquoise color scheme) to explore the distribution of RFM values and segments.


**Reasoning**:
Import necessary libraries and set the plotting style for dark background and turquoise color scheme.



In [ ]:
# Import necessary libraries for plotting / Importa as bibliotecas necessárias para plotagem
import matplotlib.pyplot as plt
import seaborn as sns

# Set the plotting style to a dark background / Define o estilo de plotagem para um fundo escuro
plt.style.use('dark_background')

# Define a turquoise color palette / Define uma paleta de cores turquesa
turquoise_palette = sns.color_palette("mako", n_colors=5) # Using 'mako' as it has turquoise shades

# Set the color palette for seaborn plots / Define a paleta de cores para os gráficos do seaborn
sns.set_palette(turquoise_palette)

**Reasoning**:
Create histograms for Recency_Days, Frequency, and Monetary to visualize their distributions with the defined style and color palette.



In [ ]:
# Create histograms for the distribution of RFM metrics
# Cria histogramas para a distribuição das métricas RFM

# Plotting Recency Distribution / Plotando a Distribuição de Recência
plt.figure(figsize=(10, 6)) # Define o tamanho da figura
sns.histplot(data=df, x='Recency_Days', bins=30, kde=True, color=turquoise_palette[0]) # Cria o histograma com KDE
plt.title('Distribution of Recency (Days since last purchase)', color='white') # Adiciona título
plt.xlabel('Recency (Days)', color='white') # Adiciona rótulo do eixo x
plt.ylabel('Number of Customers', color='white') # Adiciona rótulo do eixo y
plt.xticks(color='white') # Define a cor dos ticks do eixo x
plt.yticks(color='white') # Define a cor dos ticks do eixo y
plt.grid(axis='y', linestyle='--', alpha=0.7) # Adiciona grade no eixo y
plt.show() # Exibe o gráfico

# Plotting Frequency Distribution / Plotando a Distribuição de Frequência
plt.figure(figsize=(10, 6)) # Define o tamanho da figura
sns.histplot(data=df, x='Frequency', bins=30, kde=True, color=turquoise_palette[1]) # Cria o histograma com KDE
plt.title('Distribution of Frequency (Number of purchases)', color='white') # Adiciona título
plt.xlabel('Frequency', color='white') # Adiciona rótulo do eixo x
plt.ylabel('Number of Customers', color='white') # Adiciona rótulo do eixo y
plt.xticks(color='white') # Define a cor dos ticks do eixo x
plt.yticks(color='white') # Define a cor dos ticks do eixo y
plt.grid(axis='y', linestyle='--', alpha=0.7) # Adiciona grade no eixo y
plt.show() # Exibe o gráfico

# Plotting Monetary Distribution / Plotando a Distribuição de Valor Monetário
plt.figure(figsize=(10, 6)) # Define o tamanho da figura
sns.histplot(data=df, x='Monetary', bins=30, kde=True, color=turquoise_palette[2]) # Cria o histograma com KDE
plt.title('Distribution of Monetary Value', color='white') # Adiciona título
plt.xlabel('Monetary Value', color='white') # Adiciona rótulo do eixo x
plt.ylabel('Number of Customers', color='white') # Adiciona rótulo do eixo y
plt.xticks(color='white') # Define a cor dos ticks do eixo x
plt.yticks(color='white') # Define a cor dos ticks do eixo y
plt.grid(axis='y', linestyle='--', alpha=0.7) # Adiciona grade no eixo y
plt.show() # Exibe o gráfico

In [ ]:
# Inspect data types of the relevant columns / Inspeciona os tipos de dados das colunas relevantes
print("Data types of Frequency and Monetary:") # Imprime os tipos de dados de Frequência e Valor Monetário
display(df[['Frequency', 'Monetary']].dtypes)

# Check for unique values in Frequency and Monetary to identify potential non-numeric entries
# Verifica os valores únicos em Frequência e Valor Monetário para identificar potenciais entradas não numéricas
print("\nUnique values in Frequency:") # Imprime os valores únicos em Frequência
display(df['Frequency'].unique())

print("\nUnique values in Monetary:") # Imprime os valores únicos em Valor Monetário
display(df['Monetary'].unique())

## Present results

### Subtask:
Present the final table with customer RFM bands and a table showing the cut-off points for each band.

**Reasoning**:
Display the DataFrame with the customer RFM bands and calculate and display the cut-off points for each RFM band.

In [ ]:
# Display the final DataFrame with customer RFM bands / Exibe o DataFrame final com as faixas RFM do cliente
print("Customer RFM Bands:") # Imprime cabeçalho
display(df[['Valor_Cliente', 'R_Score', 'F_Score', 'M_Score', 'RFM_Segment', 'RFM_Score']].head()) # Exibe as primeiras linhas com as colunas RFM

# Calculate and display the cut-off points for each RFM band / Calcula e exibe os pontos de corte para cada faixa RFM
def get_band_cutoffs(data, column, num_bands):
    # Calculate quantiles / Calcula os quantis
    quantiles = data[column].quantile([i/num_bands for i in range(1, num_bands + 1)]).tolist()
    # Create a dictionary of cut-off points for each band / Cria um dicionário de pontos de corte para cada faixa
    cutoffs = {}
    cutoffs[1] = (None, quantiles[0])
    for i in range(2, num_bands + 1):
        cutoffs[i] = (quantiles[i-2], quantiles[i-1])
    return cutoffs

recency_cutoffs = get_band_cutoffs(df, 'Recency_Days', num_bands)
frequency_cutoffs = get_band_cutoffs(df, 'Frequency', num_bands)
monetary_cutoffs = get_band_cutoffs(df, 'Monetary', num_bands)

print("\nRFM Band Cut-off Points:") # Imprime cabeçalho para pontos de corte
print("Recency Cut-off Points (Days):") # Imprime cabeçalho para pontos de corte de Recência
display(recency_cutoffs)

print("\nFrequency Cut-off Points:") # Imprime cabeçalho para pontos de corte de Frequência
display(frequency_cutoffs)

print("\nMonetary Cut-off Points:") # Imprime cabeçalho para pontos de corte de Valor Monetário
display(monetary_cutoffs)

## Finish task

### Subtask:
Summarize the findings and provide insights based on the RFM analysis.

**Reasoning**:
Based on the RFM analysis, provide a summary of the customer segmentation and insights that can be drawn from the results.

**Summary and Insights:**

The RFM analysis of the customer data provides valuable insights into customer behavior. By segmenting customers based on their Recency, Frequency, and Monetary value, we can identify different customer groups with varying levels of engagement and value.

- **High-Value Customers (e.g., RFM Score 555):** These customers have made recent purchases, buy frequently, and spend a significant amount. They are your most valuable customers and should be nurtured with special offers and loyalty programs.
- **Loyal Customers (e.g., High Frequency and Monetary):** These customers buy often and spend well, but their recency might be lower. Strategies to re-engage them could be effective.
- **Recent Customers (e.g., High Recency):** These are new customers. Focus on converting them into loyal customers through targeted communication and onboarding programs.
- **Losing Customers (e.g., Low Recency, Frequency, and Monetary):** These customers have not purchased recently, buy infrequently, and spend little. Efforts might be focused on win-back campaigns or identifying why they are disengaging.

The cut-off points for each band provide clear thresholds for defining these segments. For example:

- Customers with Recency of 8 days or less fall into the highest Recency score (Band 5).
- Customers with a Frequency of more than 12 purchases fall into the highest Frequency score (Band 5).
- Customers with a Monetary value of more than 64177.4 fall into the highest Monetary score (Band 5).

By analyzing the distribution of customers across these RFM segments, businesses can tailor their marketing strategies and customer relationship management efforts to maximize customer lifetime value and improve overall business performance.

**Next Steps:**

You can further analyze each RFM segment to understand their specific characteristics and develop more targeted campaigns. For example, you could:

- Analyze the products purchased by each segment.
- Determine the preferred channels of communication for each segment.
- Develop personalized marketing messages and offers for each segment.

This RFM analysis provides a solid foundation for data-driven customer segmentation and targeted marketing.

## Save RFM Results to Excel

### Subtask:
Save the customer RFM bands and the band cut-off points to an Excel file with two separate sheets.

**Reasoning**:
Create a pandas ExcelWriter object to write multiple DataFrames to different sheets in a single Excel file. Prepare the customer RFM DataFrame and the cut-off points DataFrames. Write these DataFrames to the Excel file with appropriate sheet names.

In [ ]:
# Install the xlsxwriter library / Instala a biblioteca xlsxwriter
!pip install xlsxwriter

In [ ]:
# Create a DataFrame for the cut-off points / Cria um DataFrame para os pontos de corte
cutoffs_df = pd.DataFrame({
    'Métrica RFM': ['Recência (Dias)', 'Frequência', 'Valor Monetário'], # Métrica RFM em Português
    'RFM Metric': ['Recency (Days)', 'Frequency', 'Monetary Value'], # RFM Metric in English
    'Faixa 1 / Band 1': [recency_cutoffs[1], frequency_cutoffs[1], monetary_cutoffs[1]], # Faixa 1
    'Faixa 2 / Band 2': [recency_cutoffs[2], frequency_cutoffs[2], monetary_cutoffs[2]], # Faixa 2
    'Faixa 3 / Band 3': [recency_cutoffs[3], frequency_cutoffs[3], monetary_cutoffs[3]], # Faixa 3
    'Faixa 4 / Band 4': [recency_cutoffs[4], frequency_cutoffs[4], monetary_cutoffs[4]], # Faixa 4
    'Faixa 5 / Band 5': [recency_cutoffs[5], frequency_cutoffs[5], monetary_cutoffs[5]]  # Faixa 5
})

# Select the relevant columns for the customer RFM results / Seleciona as colunas relevantes para os resultados RFM do cliente
customer_rfm_df = df[['Valor_Cliente', 'R_Score', 'F_Score', 'M_Score', 'RFM_Segment', 'RFM_Score']]

# Define the output file path / Define o caminho do arquivo de saída
output_path = '/content/RFM_Results.xlsx'

# Create an ExcelWriter object using xlsxwriter engine / Cria um objeto ExcelWriter usando o engine xlsxwriter
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    # Write the customer RFM DataFrame to the 'Resposta' sheet / Escreve o DataFrame RFM do cliente na aba 'Resposta'
    customer_rfm_df.to_excel(writer, sheet_name='Resposta', index=False)
    # Write the cut-off points DataFrame to the 'Cut_off_Points' sheet / Escreve o DataFrame de pontos de corte na aba 'Cut_off_Points'
    cutoffs_df.to_excel(writer, sheet_name='Cut_off_Points', index=False)

    # Get the xlsxwriter workbook and worksheets objects / Obtém os objetos workbook e worksheet do xlsxwriter
    workbook  = writer.book
    worksheet_resposta = writer.sheets['Resposta']
    worksheet_cutoffs = writer.sheets['Cut_off_Points']

    # Define formats for header rows / Define formatos para as linhas de cabeçalho
    header_format_recency = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#4C78A8', # A shade of turquoise
        'border': 1
    })

    header_format_frequency = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#72B7B2', # Another shade of turquoise
        'border': 1
    })

    header_format_monetary = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#A5B8B9', # A lighter shade of turquoise/gray
        'border': 1
    })

    # Apply format to the header row of the 'Resposta' sheet / Aplica formato à linha de cabeçalho da aba 'Resposta'
    for col_num, value in enumerate(customer_rfm_df.columns):
        if value in ['Valor_Cliente']:
             worksheet_resposta.write(0, col_num, value) # No color for customer name
        elif value in ['R_Score']:
            worksheet_resposta.write(0, col_num, value, header_format_recency)
        elif value in ['F_Score']:
            worksheet_resposta.write(0, col_num, value, header_format_frequency)
        elif value in ['M_Score']:
            worksheet_resposta.write(0, col_num, value, header_format_monetary)
        elif value in ['RFM_Segment', 'RFM_Score']:
             worksheet_resposta.write(0, col_num, value) # No specific color for combined segment/score


    # Apply format to the header row of the 'Cut_off_Points' sheet / Aplica formato à linha de cabeçalho da aba 'Cut_off_Points'
    for col_num, value in enumerate(cutoffs_df.columns):
         if value in ['Métrica RFM', 'RFM Metric']:
              worksheet_cutoffs.write(0, col_num, value) # No color for metric names
         elif 'Recência' in value or 'Recency' in value:
             worksheet_cutoffs.write(0, col_num, value, header_format_recency)
         elif 'Frequência' in value or 'Frequency' in value:
             worksheet_cutoffs.write(0, col_num, value, header_format_frequency)
         elif 'Valor Monetário' in value or 'Monetary Value' in value:
             worksheet_cutoffs.write(0, col_num, value, header_format_monetary)
         else:
              worksheet_cutoffs.write(0, col_num, value)


print(f"RFM results with colored headers saved to {output_path}") # Imprime mensagem de confirmação